In [1]:
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/code/bm_preproc.py

SYSTEM_WGETRC = c:/progra~1/wget/etc/wgetrc
syswgetrc = C:\Program Files (x86)\GnuWin32/etc/wgetrc
--2021-02-27 22:10:17--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/code/bm_preproc.py
Resolving d28rh4a8wq0iu5.cloudfront.net... 13.225.143.199, 13.225.143.44, 13.225.143.82, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net|13.225.143.199|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9400 (9.2K) [application/octet-stream]
Saving to: `bm_preproc.py'

     0K .........                                             100% 1.75M=0.005s

2021-02-27 22:10:17 (1.75 MB/s) - `bm_preproc.py' saved [9400/9400]



In [2]:
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta

SYSTEM_WGETRC = c:/progra~1/wget/etc/wgetrc
syswgetrc = C:\Program Files (x86)\GnuWin32/etc/wgetrc
--2021-02-27 22:10:36--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
Resolving d28rh4a8wq0iu5.cloudfront.net... 13.225.143.199, 13.225.143.44, 13.225.143.82, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net|13.225.143.199|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 810105 (791K) [application/octet-stream]
Saving to: `chr1.GRCh38.excerpt.fasta'

     0K .......... .......... .......... .......... ..........  6%  309K 2s
    50K .......... .......... .......... .......... .......... 12% 1.04M 1s
   100K .......... .......... .......... .......... .......... 18% 2.97M 1s
   150K .......... .......... .......... .......... .......... 25%  629K 1s
   200K .......... .......... .......... .......... .......... 31% 1.30M 1s
   250K .......... .......... .......... .......... .......... 37% 1.67M 1s
   300K .......... .......... .......... 

In [5]:
def readGenome(filename):
    genome = ""

    with open(filename, "r") as f:
        for line in f:
            if not line[0] == ">":
                genome += line.rstrip()

    return genome

def reverseComplement(s):
    complement = ""
    complements = {"A": "T", "C": "G", "G": "C", "T": "A", "N": "N"}
    for base in s:
        # complement += complements[base] would yield just the complement
        complement = complements[base] + complement
    return complement

def naive(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):
        match = True
        for j in range(len(p)):
            if t[i+j] != p[j]:
                match = False
                break
        if match:
            occurrences.append(i)
    return occurrences

def naive_with_counts(p, t):
    occurrences = []
    num_alignments = 0
    num_character_comparisons = 0
    for i in range(len(t) - len(p) + 1):
        num_alignments += 1
        match = True
        for j in range(len(p)):
            num_character_comparisons += 1
            if t[i+j] != p[j]:
                match = False
                break
        if match:
            occurrences.append(i)
    return (occurrences, num_alignments, num_character_comparisons)

# p = 'word'
# t = 'there would have been a time for such a word'
p = 'needle'
t = 'needle need noodle needle'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(f"occurrences, num_alignments, num_character_comparisons: {occurrences, num_alignments, num_character_comparisons}")

occurrences, num_alignments, num_character_comparisons: ([0, 19], 20, 35)


In [3]:
from bm_preproc import BoyerMoore

def boyer_moore(p, p_bm, t):
    """p is pattern, p_bm is Boyer-Moore object, t is the text we want to search in"""
    i = 0
    occurrences = []
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False  # update as we go
        for j in range(len(p) - 1, -1, -1):
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    
    return occurrences

def boyer_moore_with_counts(p, p_bm, t):
    i = 0
    occurrences = []
    num_alignments = 0
    num_character_comparisons = 0
    while i < len(t) - len(p) + 1:
        num_alignments += 1
        shift = 1
        mismatched = False  # update as we go
        for j in range(len(p) - 1, -1, -1):
            num_character_comparisons += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return (occurrences, num_alignments, num_character_comparisons)

# p = 'word'
# t = 'there would have been a time for such a word'
# lowercase_alphabet = 'abcdefghijklmnopqrstuvwxyz '
# p_bm = BoyerMoore(p, lowercase_alphabet)
p = 'needle'
t = 'needle need noodle needle'
lowercase_alphabet = 'abcdefghijklmnopqrstuvwxyz '
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(f"occurrences, num_alignments, num_character_comparisons: {occurrences, num_alignments, num_character_comparisons}")

occurrences, num_alignments, num_character_comparisons: ([0, 19], 5, 18)


In [6]:
# questions 1 and 2
def readGenome(filename):
    genome = ""

    with open(filename, "r") as f:
        for line in f:
            if not line[0] == ">":
                genome += line.rstrip()

    return genome

p = "GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG"
t = readGenome("chr1.GRCh38.excerpt.fasta")
p_bm = BoyerMoore(p)
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(f"occurrences, num_alignments, num_character_comparisons: {occurrences, num_alignments, num_character_comparisons}")

occurrences, num_alignments, num_character_comparisons: ([56922], 799954, 984143)


In [7]:
# question 3

def readGenome(filename):
    genome = ""

    with open(filename, "r") as f:
        for line in f:
            if not line[0] == ">":
                genome += line.rstrip()

    return genome

p = "GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG"
t = readGenome("chr1.GRCh38.excerpt.fasta")
p_bm = BoyerMoore(p)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(f"occurrences, num_alignments, num_character_comparisons: {occurrences, num_alignments, num_character_comparisons}")

occurrences, num_alignments, num_character_comparisons: ([56922], 127974, 165191)


In [28]:
# question 4

# Implement the pigeonhole principle using Index to find exact matches for the partitions. 
# Assume P always has length 24, and that we are looking for approximate matches with up to 2 mismatches (substitutions).
# We will use an 8-mer index.

# Download the Python module for building a k-mer index. 

# https://d28rh4a8wq0iu5.cloudfront.net/ads1/code/kmer_index.py

# Write a function that, given a length-24 pattern P and given an Index object built on 8-mers,
# finds all approximate occurrences of P within T with up to 2 mismatches. Insertions and deletions are not allowed. 
# Don't consider any reverse complements.

# How many times does the string GGCGCGGTGGCTCACGCCTGTAAT, which is derived from a human Alu sequence, 
# occur with up to 2 substitutions in the excerpt of human chromosome 1?  (Don't consider reverse complements here.)

# Hint 1: Multiple index hits might direct you to the same match multiple times,
#         but be careful not to count a match more than once.

# Hint 2: You can check your work by comparing the output of your new function to that of the 
#         naive_2mm function implemented in the previous module.

!wget https://d28rh4a8wq0iu5.cloudfront.net/ads1/code/kmer_index.py

SYSTEM_WGETRC = c:/progra~1/wget/etc/wgetrc
syswgetrc = C:\Program Files (x86)\GnuWin32/etc/wgetrc
--2021-02-28 14:13:18--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/code/kmer_index.py
Resolving d28rh4a8wq0iu5.cloudfront.net... 13.226.211.214, 13.226.211.176, 13.226.211.37, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net|13.226.211.214|:443... connected.
ERROR: cannot verify d28rh4a8wq0iu5.cloudfront.net's certificate, issued by `/C=US/O=DigiCert Inc/CN=DigiCert Global CA G2':
  Unable to locally verify the issuer's authority.
To connect to d28rh4a8wq0iu5.cloudfront.net insecurely, use `--no-check-certificate'.
Unable to establish SSL connection.


In [8]:
# question 4 continued

import bisect

class Index(object):
    """ Holds a substring index for a text T """

    def __init__(self, t, k):
        """ Create index from all substrings of t of length k """
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer

    def query(self, p):
        """ Return index hits for first k-mer of p """
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

def queryIndex(p, t, index):
    k = index.k
    offsets = []
    for i in index.query(p):
        if p[k:] == t[i+k:i+len(p)]:  # verifying that p at that offset matches t entirely
            offsets.append(i)
    return offsets

def approximate_matching(p, t, n):
    """Function takes pattern p and text t. It finds all indices at which p matches t with up to n mismatches."""
    segment_length = int(round(len(p) / (n + 1)))  # without int, we could get a float number
    all_matches = set()
    for i in range(n + 1):
        start = i * segment_length
        end = min((i + 1) * segment_length, len(p))
        p_bm = BoyerMoore(p[start:end], alphabet="ACGT")
        matches = boyer_moore(p[start:end], p_bm, t)
        
        for m in matches:
            if m < start or m - start + len(p) > len(t):
                continue  # skip the rest of this loop

            mismatches = 0
            for j in range(0, start):  # loop through the left side of the partition
                if p[j] != t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):  # loop through the right side of the partition
                if p[j] != t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break

            if mismatches <= n:
                all_matches.add(m - start)
    
    return list(all_matches)

p = "GGCGCGGTGGCTCACGCCTGTAAT"
t = readGenome("chr1.GRCh38.excerpt.fasta")
n = 2
occurrences = approximate_matching(p, t, n)
print(len(occurrences))

19


In [42]:
# question 5

# Using the instructions given in Question 4, how many total index hits are there when searching for occurrences of 
# GGCGCGGTGGCTCACGCCTGTAAT with up to 2 substitutions in the excerpt of human chromosome 1?
# (Don't consider reverse complements.)

# Hint: You should be able to use the boyer_moore function (or the slower naive function) to double-check your answer.

import bisect

class Index(object):
    """ Holds a substring index for a text T """

    def __init__(self, t, k):
        """ Create index from all substrings of t of length k """
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer

    def query(self, p):
        """ Return index hits for first k-mer of p """
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

def queryIndex(p, t, index):
    k = index.k
    offsets = []
    for i in index.query(p):
        if p[k:] == t[i+k:i+len(p)]:  # verifying that p at that offset matches t entirely
            offsets.append(i)
    print(f"queryIndex: len(index.query(p)) = {len(index.query(p))}")
    return offsets

def approximate_matching(p, t, n):
    """Function takes pattern p and text t. It finds all indices at which p matches t with up to n mismatches."""
    segment_length = int(round(len(p) / (n + 1)))  # without int, we could get a float number
    all_matches = set()
    for i in range(n + 1):
        start = i * segment_length
        end = min((i + 1) * segment_length, len(p))
        p_bm = BoyerMoore(p[start:end], alphabet="ACGT")
        matches = boyer_moore(p[start:end], p_bm, t)
        
        for m in matches:
            if m < start or m - start + len(p) > len(t):
                continue  # skip the rest of this loop

            mismatches = 0
            for j in range(0, start):  # loop through the left side of the partition
                if p[j] != t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):  # loop through the right side of the partition
                if p[j] != t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break

            if mismatches <= n:
                all_matches.add(m - start)
    
    return list(all_matches)

p = "GGCGCGGTGGCTCACGCCTGTAAT"
t = readGenome("chr1.GRCh38.excerpt.fasta")
k = 8
n = 2
index = Index(t, k)
offsets = queryIndex(p, t, index)
print(f"offsets = {offsets}")
all_matches = approximate_matching(p, t, n)
print(f"len(all_matches) = {len(all_matches)}")

queryIndex: len(index.query(p)) = 13
offsets = [56922, 262042, 364263, 657496, 717706]
len(all_matches) = 19


In [49]:
import bisect

class Index(object):
    """ Holds a substring index for a text T """

    def __init__(self, t, k):
        """ Create index from all substrings of t of length k """
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer

    def query(self, p):
        """ Return index hits for first k-mer of p """
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits
    
p = "GGCGCGGTGGCTCACGCCTGTAAT"
t = readGenome("chr1.GRCh38.excerpt.fasta")
k = 8

index = Index(t, k)

print(f"first element: {index.query(p[0:])}")
print(f"first element: {len(index.query(p[0:]))}")
print(f"second element: {index.query(p[1:])}")
print(f"second element: {len(index.query(p[1:]))}")
print(f"third element: {index.query(p[2:])}")
print(f"third element: {len(index.query(p[2:]))}")

first element: [56922, 57056, 83720, 84641, 147558, 160729, 191452, 262042, 364263, 657496, 681737, 717706, 725061]
first element: 13
second element: [56923, 57057, 83721, 84642, 147559, 160730, 191453, 262043, 364264, 429300, 657497, 681738, 717707, 725062]
second element: 14
third element: [56924, 57058, 84643, 147560, 160731, 191454, 262044, 359294, 364265, 429301, 657498, 681739, 717708, 725063]
third element: 14


In [41]:
from bm_preproc import BoyerMoore

def boyer_moore_with_counts(p, p_bm, t):
    i = 0
    occurrences = []
    num_alignments = 0
    num_character_comparisons = 0
    while i < len(t) - len(p) + 1:
        num_alignments += 1
        shift = 1
        mismatched = False  # update as we go
        for j in range(len(p) - 1, -1, -1):
            num_character_comparisons += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return (occurrences, num_alignments, num_character_comparisons)

def boyer_moore(p, p_bm, t):
    """p is pattern, p_bm is Boyer-Moore object, t is the text we want to search in"""
    i = 0
    occurrences = []
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False  # update as we go
        for j in range(len(p) - 1, -1, -1):
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    
    return occurrences

def boyer_moore_2mismatches(p, p_bm, t, n):
    """p is pattern, p_bm is Boyer-Moore object, t is the text we want to search in, n is number of mismatches"""
    i = 0
    occurrences = []
    while i < len(t) - len(p) + 1:
        shift = 1
        n_mismatches = 0
        for j in range(len(p) - 1, -1, -1):
            if p[j] != t[i+j]:
                n_mismatches += 1
                if n_mismatches > n:
                    skip_bc = p_bm.bad_character_rule(j, t[i+j])
                    skip_gs = p_bm.good_suffix_rule(j)
                    shift = max(shift, skip_bc, skip_gs)
                    # print(f"boyer_moore_2mismatches: shift = {shift}")
                    break
            pass
        if n_mismatches <= n:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    
    return occurrences

def naive_2mismatches(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):
        match = True
        n_mismatches = 0
        for j in range(len(p)):
            if t[i+j] != p[j]:
                # match = False
                n_mismatches += 1
                if n_mismatches > 2:
                    match = False
                    break
        if match:
            occurrences.append(i)
    return occurrences

p = "GGCGCGGTGGCTCACGCCTGTAAT"
t = readGenome("chr1.GRCh38.excerpt.fasta")
p_bm = BoyerMoore(p)
n = 2
occurrences = boyer_moore(p, p_bm, t)
print(f"boyer_moore: len(occurrences) = {len(occurrences)}")

total_hits = naive_2mismatches(p, t)
print(f"naive_2mismatches: len(total_hits) = {len(total_hits)}")

hits = boyer_moore_2mismatches(p, p_bm, t, n)
print(f"boyer_moore_2mismatches: len(hits) = {len(hits)}")

hits = boyer_moore_2mismatches(p, p_bm, t, 0)
print(f"boyer_moore_2mismatches n = 0: len(hits) = {len(hits)}")

boyer_moore: len(occurrences) = 5
naive_2mismatches: len(total_hits) = 19
boyer_moore_2mismatches: len(hits) = 0
boyer_moore_2mismatches n = 0: len(hits) = 5


In [12]:
# question 6

# Let's examine whether there is a benefit to using an index built using subsequences of T rather than substrings, 
# as we discussed in the "Variations on k-mer indexes" video.  We'll consider subsequences involving every N characters.
# For example, if we split ATATAT into two substring partitions, we would get partitions ATA (the first half) 
# and TAT (second half).  But if we split ATATAT into two  subsequences  by taking every other character, we would get AAA
# (first, third and fifth characters) and TTT (second, fourth and sixth).

# Another way to visualize this is using numbers to show how each character of P is allocated to a partition.
# Splitting a length-6 pattern into two substrings could be represented as 111222, and 
# splitting into two subsequences of every other character could be represented as 121212

# The following class SubseqIndex is a more general implementation of Index that additionally handles subsequences.
# It only considers subsequences that take every Nth character.

# Write a function that, given a length-24 pattern P and given a SubseqIndex object built with k = 8 and ival = 3,
# finds all approximate occurrences of P within T with up to 2 mismatches.

# When using this function, how many total index hits are there when searching for GGCGCGGTGGCTCACGCCTGTAAT with up to
# 2 substitutions in the excerpt of human chromosome 1?  (Again, don't consider reverse complements.)

import bisect
   
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

ind = SubseqIndex('ATATAT', 3, 2)
print(ind.index)
p = 'TTATAT'
print(ind.query(p[0:]))
print(ind.query(p[1:]))

[('AAA', 0), ('TTT', 1)]
[]
[1]


In [21]:
# question 6 continued
# Write a function that, given a length-24 pattern P and given a SubseqIndex object built with k = 8 and ival = 3,
# finds all approximate occurrences of P within T with up to 2 mismatches.

# When using this function, how many total index hits are there when searching for GGCGCGGTGGCTCACGCCTGTAAT with up to
# 2 substitutions in the excerpt of human chromosome 1?  (Again, don't consider reverse complements.)

import bisect
   
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits
    
def queryIndex(p, t, index):
    k = index.k
    offsets = []
    for i in index.query(p):
        if p[k:] == t[i+k:i+len(p)]:  # verifying that p at that offset matches t entirely
            offsets.append(i)
    print(len(index.query(p)))
    return offsets

def approximate_matching(p, t, n):
    """Function takes pattern p and text t. It finds all indices at which p matches t with up to n mismatches."""
    segment_length = int(round(len(p) / (n + 1)))  # without int, we could get a float number
    all_matches = set()
    for i in range(n + 1):
        start = i * segment_length
        end = min((i + 1) * segment_length, len(p))
        p_bm = BoyerMoore(p[start:end], alphabet="ACGT")
        matches = boyer_moore(p[start:end], p_bm, t)
        
        for m in matches:
            if m < start or m - start + len(p) > len(t):
                continue  # skip the rest of this loop

            mismatches = 0
            for j in range(0, start):  # loop through the left side of the partition
                if p[j] != t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):  # loop through the right side of the partition
                if p[j] != t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break

            if mismatches <= n:
                all_matches.add(m - start)
    
    return list(all_matches), len(list(all_matches))

# p = "GGCGCGGTGGCTCACGCCTGTAAT"
# t = readGenome("chr1.GRCh38.excerpt.fasta")
# k = 8
# ival = 3
# n = 2

p = "GGCGCGGTGGCTCACGCCTGTAAT"
t = readGenome("chr1.GRCh38.excerpt.fasta")
k = 8
index = SubseqIndex(t, k, ival)
queryIndex(p, t, index)

# t = 'to-morrow and to-morrow and to-morrow creeps in this petty pace'
# p = 'to-morrow and to-morrow '
# subseq_ind = SubseqIndex(t, 8, 3)
# occurrences, num_index_hits = SubseqIndex.query(p, t, subseq_ind)
# print(f"occurrences, num_index_hits: {occurrences, num_index_hits}")

35


[56922, 262042, 273669, 364263, 465647, 657496, 717706]